# Backtesting Evaluation

This notebook evaluates the accuracy and performance of the three-agent system of banks made in this repository with real world data.

# Prerequisites

This notebook uses the Federal Reserve Economic Data (FRED) API to import economic data and requires a `.env` file to load the `FRED_API_KEY`. Make sure the .env file is present in the root directory of the repository and includes the key before running the notebook.

In [ ]:
import logging
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display

from agentomics.common.data_structures import ThreeBankGlobalState
from agentomics.common.types import NonnegPercent, Percent
from agentomics.utils.fred_download import get_fred_data
from agentomics.utils.fred_post_processing import get_longest_common_date_range
from agentomics.utils.logging import configure_logging
from scripts.economic_simulations.three_banks import simulate_two_way

load_dotenv()
configure_logging(log_to_console=False, log_to_file=True, log_level=logging.INFO)

OUTPUT_CSV_PATH = "~/ode/agentomics/output/notebook_output.csv"
MODEL_NAME = "groq/llama-3.1-70b-versatile"

## Gathering Input Data

Gather data for [US GDP Growth Rate](https://fred.stlouisfed.org/series/A191RL1Q225SBEA), [US Unemployment Rate](https://fred.stlouisfed.org/series/UNRATE), and [US Consumer Price Index](https://fred.stlouisfed.org/series/CPIAUCSL) from the FRED database.

In [ ]:
# FRED API details
fred_api_url = "https://api.stlouisfed.org/fred/series/observations"
fred_api_key = os.getenv("FRED_API_KEY")
fred_gdp_series_id = "A191RL1Q225SBEA"  # Please verify this ID is still valid
fred_unemployment_series_id = "UNRATE"  # Please verify this ID is still valid
fred_cpi_series_id = "CPIAUCSL"  # Please verify this ID is still valid
frequency = "q"  # Quarterly frequency
observation_start = "2022-01-01"  # Start date for all economic series
observation_end = "2024-01-01"  # End date for all economic series

### Download Economic Data

In [ ]:
gdp_df = get_fred_data(fred_gdp_series_id, frequency, observation_start, observation_end, "Quarterly GDP Growth Rate")

# Convert percentages to decimals
gdp_df["Quarterly GDP Growth Rate"] /= 100.0

unemployment_df = get_fred_data(fred_unemployment_series_id, frequency, observation_start, observation_end, "Quarterly Unemployment Rate")

# Convert percentages to decimals
unemployment_df["Quarterly Unemployment Rate"] /= 100.0

cpi_df = get_fred_data(fred_cpi_series_id, frequency, observation_start, observation_end, "Quarterly CPI")

inflation_df = pd.DataFrame()
inflation_df["date"] = cpi_df["date"]
inflation_df["Quarterly Inflation Rate"] = cpi_df["Quarterly CPI"].pct_change()

display(inflation_df)

### Graph Economic Data

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Top Left: GDP
axs[0, 0].plot(gdp_df["date"], gdp_df["Quarterly GDP Growth Rate"], marker="o", linestyle="-", color="blue")
axs[0, 0].set_title("Quarterly GDP Growth Rate")
axs[0, 0].set_xlabel("Date")
axs[0, 0].set_ylabel("GDP Growth Rate")
axs[0, 0].grid(True)
axs[0, 0].tick_params(axis="x", rotation=45)

# Top Right: Unemployment
axs[0, 1].plot(unemployment_df["date"], unemployment_df["Quarterly Unemployment Rate"], marker="o", linestyle="-", color="red")
axs[0, 1].set_title("Quarterly Unemployment Rate")
axs[0, 1].set_xlabel("Date")
axs[0, 1].set_ylabel("Unemployment Rate")
axs[0, 1].grid(True)
axs[0, 1].tick_params(axis="x", rotation=45)

# Bottom Left: CPI
axs[1, 0].plot(cpi_df["date"], cpi_df["Quarterly CPI"], marker="o", linestyle="-", color="green")
axs[1, 0].set_title("Quarterly CPI")
axs[1, 0].set_xlabel("Date")
axs[1, 0].set_ylabel("CPI")
axs[1, 0].grid(True)
axs[1, 0].tick_params(axis="x", rotation=45)

# Bottom Right: Inflation
axs[1, 1].plot(inflation_df["date"], inflation_df["Quarterly Inflation Rate"], marker="o", linestyle="--", color="purple")
axs[1, 1].set_title("Quarterly Inflation Rate")
axs[1, 1].set_xlabel("Date")
axs[1, 1].set_ylabel("Inflation Rate")
axs[1, 1].grid(True)
axs[1, 1].tick_params(axis="x", rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the combined plot
plt.show()

## Cleaning input data

Select the longest continuous block from all series that does not contain any NaNs. This will conform to what the simulation expects while ensuring that we have no NaN values.

### Get Cleaned Input Data

In [ ]:
clean_df = get_longest_common_date_range([gdp_df, unemployment_df, cpi_df, inflation_df])

display(clean_df)

### Graph Cleaned Data as Confirmation

In [ ]:
# Create a figure with 4 subplots arranged in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Plot Quarterly GDP Growth Rate
axs[0, 0].plot(clean_df["date"], clean_df["Quarterly GDP Growth Rate"],
               marker="o", linestyle="-", color="blue")
axs[0, 0].set_title("Quarterly GDP Growth Rate")
axs[0, 0].set_xlabel("Date")
axs[0, 0].set_ylabel("GDP Growth Rate (%)")
axs[0, 0].grid(True)
axs[0, 0].tick_params(axis="x", rotation=45)

# Plot Quarterly Unemployment Rate
axs[0, 1].plot(clean_df["date"], clean_df["Quarterly Unemployment Rate"],
               marker="o", linestyle="-", color="red")
axs[0, 1].set_title("Quarterly Unemployment Rate")
axs[0, 1].set_xlabel("Date")
axs[0, 1].set_ylabel("Unemployment Rate (%)")
axs[0, 1].grid(True)
axs[0, 1].tick_params(axis="x", rotation=45)

# Plot Quarterly Inflation Rate
axs[1, 0].plot(clean_df["date"], clean_df["Quarterly Inflation Rate"],
               marker="o", linestyle="-", color="green")
axs[1, 0].set_title("Quarterly Inflation Rate")
axs[1, 0].set_xlabel("Date")
axs[1, 0].set_ylabel("Inflation Rate (%)")
axs[1, 0].grid(True)
axs[1, 0].tick_params(axis="x", rotation=45)

# Plot CPI Readings
axs[1, 1].plot(clean_df["date"], clean_df["Quarterly CPI"],
               marker="o", linestyle="-", color="purple")
axs[1, 1].set_title("Quarterly CPI")
axs[1, 1].set_xlabel("Date")
axs[1, 1].set_ylabel("CPI")
axs[1, 1].grid(True)
axs[1, 1].tick_params(axis="x", rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Ensure 'date' columns are in datetime format
for df in [gdp_df, unemployment_df, inflation_df, cpi_df, clean_df]:
    df["date"] = pd.to_datetime(df["date"])

# Determine the overall date range
all_dates = pd.concat([
    gdp_df["date"],
    unemployment_df["date"],
    inflation_df["date"],
    cpi_df["date"],
    clean_df["date"]
])

min_date = all_dates.min()
max_date = all_dates.max()

# Create a common date range for x-ticks (e.g., quarterly)
common_dates = pd.date_range(start=min_date, end=max_date, freq="QS")  # 'QS' stands for Quarter Start

# Variables to plot
variables = [
    {
        "name": "Quarterly GDP Growth Rate",
        "before_df": gdp_df,
        "before_col": "Quarterly GDP Growth Rate",
        "after_col": "Quarterly GDP Growth Rate",
        "ylabel": "GDP Growth Rate (%)",
        "color": "blue"
    },
    {
        "name": "Quarterly Unemployment Rate",
        "before_df": unemployment_df,
        "before_col": "Quarterly Unemployment Rate",
        "after_col": "Quarterly Unemployment Rate",
        "ylabel": "Unemployment Rate (%)",
        "color": "red"
    },
    {
        "name": "Quarterly Inflation Rate",
        "before_df": inflation_df,
        "before_col": "Quarterly Inflation Rate",
        "after_col": "Quarterly Inflation Rate",
        "ylabel": "Inflation Rate (%)",
        "color": "green"
    },
    {
        "name": "Quarterly CPI",
        "before_df": cpi_df,
        "before_col": "Quarterly CPI",
        "after_col": "Quarterly CPI",
        "ylabel": "CPI",
        "color": "purple"
    }
]

# Create a figure
fig = plt.figure(figsize=(20, 15))

# Create an outer GridSpec with 2 rows and 2 columns
outer_grid = gridspec.GridSpec(2, 2, wspace=0.15, hspace=0.3)

for idx, var in enumerate(variables):
    # Calculate row and column indices
    row = idx // 2
    col = idx % 2

    # Create an inner GridSpec within the current outer grid cell
    inner_grid = gridspec.GridSpecFromSubplotSpec(
        1, 2, subplot_spec=outer_grid[idx], wspace=0.05
    )

    # Extract data and column names
    before_df = var["before_df"]
    before_col = var["before_col"]
    after_col = var["after_col"]
    ylabel = var["ylabel"]
    color = var["color"]
    name = var["name"]

    # Before and After Axes, sharing y-axis
    ax_before = fig.add_subplot(inner_grid[0])
    ax_after = fig.add_subplot(inner_grid[1], sharey=ax_before)

    # Plot Before Cleaning Data
    ax_before.plot(
        before_df["date"], before_df[before_col],
        linestyle="-", marker="o", color=color
    )
    ax_before.set_title(f"{name} (Before)")
    ax_before.set_xlabel("Date")
    ax_before.set_ylabel(ylabel)
    ax_before.grid(True)
    ax_before.tick_params(axis="x", rotation=45)

    # Plot After Cleaning Data
    ax_after.plot(
        clean_df["date"], clean_df[after_col],
        linestyle="-", marker="o", color=color
    )
    ax_after.set_title(f"{name} (After)")
    ax_after.set_xlabel("Date")
    ax_after.grid(True)
    ax_after.tick_params(axis="x", rotation=45)

    # Set the same x-axis limits and ticks for both subplots
    for ax in [ax_before, ax_after]:
        ax.set_xlim([min_date, max_date])
        ax.set_xticks(common_dates)
        # Correctly format the date labels with year and quarter
        ax.set_xticklabels([f"{date.year}-Q{((date.month - 1)//3 + 1)}" for date in common_dates], rotation=45)
        ax.xaxis.set_major_formatter(plt.NullFormatter())  # Hide default x-axis labels
        ax.xaxis.set_minor_formatter(plt.NullFormatter())

    # Remove y-tick labels on the right subplot to avoid duplication
    plt.setp(ax_after.get_yticklabels(), visible=False)

# Adjust layout to prevent overlap
# plt.tight_layout()

# Optionally, adjust the x-axis labels after tight_layout
for ax in fig.get_axes():
    ax.set_xlabel("Date")
    ax.set_xticks(common_dates)
    ax.set_xticklabels([f"{date.year}-Q{((date.month - 1)//3 + 1)}" for date in common_dates], rotation=45)

# Display the plot
plt.show()

In [ ]:
# Set up data to put into simulation
N = len(clean_df)
K = 5  # We take the first 5 rows as context for the economic variables

sub_df = clean_df.head(K)
gdp_input = list(map(Percent, sub_df["Quarterly GDP Growth Rate"].tolist()))
unemployment_input = list(map(NonnegPercent, sub_df["Quarterly Unemployment Rate"].tolist()))
inflation_input = list(map(Percent, sub_df["Quarterly Inflation Rate"].tolist()))
# Set all knobs to N/A [== -infinity for the custom types]
target_interest_rate_input = [NonnegPercent(float("-inf"))] * K
securities_holdings_pc_change_input = [Percent(float("-inf"))] * K
loan_to_deposit_ratio_input = [NonnegPercent(float("-inf"))] * K
deposit_interest_rate_input = [NonnegPercent(float("-inf"))] * K
loans_interest_rate_input = [NonnegPercent(float("-inf"))] * K
consumer_loan_focus_input = [NonnegPercent(float("-inf"))] * K

In [ ]:
globals = ThreeBankGlobalState()
globals.economic_variables.gdp_growth_rate.set_array(gdp_input)
globals.economic_variables.unemployment_rate.set_array(unemployment_input)
globals.economic_variables.inflation_rate.set_array(inflation_input)
globals.central_bank_knobs.target_interest_rate.set_array(target_interest_rate_input)
globals.central_bank_knobs.securities_holdings_pc_change.set_array(securities_holdings_pc_change_input)
globals.big_bank_knobs.loan_to_deposit_ratio.set_array(loan_to_deposit_ratio_input)
globals.big_bank_knobs.deposit_interest_rate.set_array(deposit_interest_rate_input)
globals.small_bank_knobs.loans_interest_rate.set_array(loans_interest_rate_input)
globals.small_bank_knobs.consumer_loan_focus.set_array(consumer_loan_focus_input)
globals.number_of_quarters_to_simulate = N - K

simulate_two_way(globals, MODEL_NAME, outfile="~/ode/agentomics/output/notebook_output.csv")

globals_pd = globals.to_pandas_df()
globals_pd.to_csv(OUTPUT_CSV_PATH)

In [ ]:
globals_pd = pd.read_csv(OUTPUT_CSV_PATH)
display(globals_pd)

In [ ]:
# Combine the two series into a DataFrame, aligning on the index
join_df = clean_df.join(globals_pd)

display(join_df)

compare_gdp_df = join_df[["gdp_growth_rate", "Quarterly GDP Growth Rate"]].dropna()


plt.figure(figsize=(12, 6))

# Plot the predicted GDP growth rate
plt.plot(join_df.index, join_df["gdp_growth_rate"], label="Predicted GDP Growth Rate", linestyle="--", marker="o")

# Plot the true GDP growth rate
plt.plot(join_df.index, join_df["Quarterly GDP Growth Rate"], label="True GDP Growth Rate", linestyle="-", marker="x")

# Customize the plot
plt.title("Predicted vs True GDP Growth Rate")
plt.xlabel("Time")
plt.ylabel("GDP Growth Rate (%)")
plt.legend()
plt.grid(True)

# Rotate x-axis labels if they are too crowded (e.g., dates or quarters)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()